# PLAYGROUND NOTEBOOK
Notebook untuk mengecek kode yang telah dibuat

In [1]:
import pandas as pd
from utils import load_model_and_tokenizer
pd.set_option('display.max_colwidth', None)

In [2]:
llama32_results = pd.read_json("meta-llama_Llama-3.2-1B-Instruct-50_results.json")
model_name = "meta-llama/Llama-3.2-1B-Instruct"
model, tokenizer, config = load_model_and_tokenizer(model_name)

Loading model on cuda with torch.float16


## Cek Prioritas Memilih Token *Ground-Truth*

### Explore apakah ada token x di dalam ground truth yang mana token x tersebut dihasilkan berkali-kali di *generated answer*

menyimpan rows tersebut ke variabel *rows_with_duplicates*

#### meta-llama_Llama-3.2-1B-Instruct-50_results (NEW version)

In [3]:
from utils import is_groundtruth_duplicated_in_generation

# Tambahkan kolom baru untuk menandai apakah ada duplikasi kata, apakah kata tersebut ada di 'answer',
# dan daftar kata yang merupakan duplikat
llama32_results['duplicates_in_answer'], llama32_results['duplicate_words_list'] = zip(
    *llama32_results.apply(lambda row: is_groundtruth_duplicated_in_generation(row['generated_completion'], row['answer']), axis=1)
)

# Filter baris yang memiliki duplikasi kata
rows_with_duplicates = llama32_results[llama32_results['duplicates_in_answer']]
print(f"Banyak row yang ground truth terduplikat dalam generation: {rows_with_duplicates.shape[0]}")

In [10]:
rows_with_duplicates[['id', 'question', 'passage', 'generated_completion', 'answer','tokens', 'duplicate_words_list']].head(3)

,id,question,passage,generated_completion,answer,tokens,duplicate_words_list
1,-6515196497017052077-5,Siapa arsitek Balai Kota Seoul?,"Pada tanggal 18 Februari 2008, desain Yoo Kerl dari iArc terpilih dalam kompetisi desain Pemerintah Metropolitan Seoul.[6] Bagian atas depan bangunan baru ini dirancang untuk menonjol dalam bentuk lengkung. Perancang Yoo merangkul tiga kata kunci yakni ""tradisi, warga negara, dan masa depan"" dalam rancangannya, yang mencerminkan elemen horisontal dari arsitektur tradisional tingkat rendah Korea dan penafsiran ulang dari nuansa mendalam dan lekukan atap atap.[7]",Arsitek yang dirancang desain Yoo Kerl dari iArc adalah Yoo Kerl.,Yoo Kerl,"[Ar, site, k, Ġyang, Ġdir, anc, ang, Ġdes, ain, ĠY, oo, ĠKer, l, Ġdari, Ġi, Arc, Ġadalah, ĠY, oo, ĠKer, l, ., <|eot_id|>]","[Yoo, Kerl]"
4,7410193178422667195-3,Dimana Jamie Richard Vardy lahir?,"Lahir di Sheffield, South Yorkshire, Vardy memulai kariernya di tim muda Stocksbridge Park Steels pada umur 16 tahun setelah dilepas Sheffield Wednesday. Ia masuk ke tim reserve dan tim utama dibawah arahan Gary Marrow tahun 2007, menerima £30 perminggu di klub. Setelah performa menakjubkannya, beberapa tim dari Football League tertarik dan tahun 2009 ia menghabiskan seminggu melakukan trial dengan Crewe Alexandra. Transfer itu tidak terjadi dan ia lalu menolak kontrak jangka pendek dengan Rotherham United.","Jamie Richard Vardy lahir di Sheffield, South Yorkshire, Inggris.","Sheffield, South Yorkshire","[J, am, ie, ĠRichard, ĠV, ardy, Ġla, hir, Ġdi, ĠSheffield, ,, ĠSouth, ĠYorkshire, ,, ĠIng, gr, is, ., <|eot_id|>]","[,]"
5,7176612552296612393-4,siapakah orang tua John Fitzgerald Kennedy?,"John Fitzgerald Kennedy lahir di 83 Beals Street, Brookline, Massachusetts, pada tanggal 29 Mei 1917[10] dari pasangan pebisnis/politikus Joseph Patrick ""Joe"" Kennedy, Sr. (1888–1969) dan filantropis Rose Elizabeth Fitzgerald (1890–1995). Joe adalah putra sulung pebisnis/politikus Patrick Joseph ""P. J."" Kennedy (1858–1929) dan Mary Augusta Hickey (1857–1923). Rose adalah putri sulung Wali Kota Boston John Francis ""Honey Fitz"" Fitzgerald (1863–1950) dan Mary Josephine ""Josie"" Hannon (1865–1964). Keempat kakek-neneknya adalah anak-anak imigran Irlandia.[1]","Menurut teks, Joseph Patrick ""Joe"" Kennedy, Sr. adalah ayah John Fitzgerald Kennedy, sedangkan Rose Elizabeth Fitzgerald adalah ibu.","Joseph Patrick ""Joe"" Kennedy, Sr. (1888–1969) dan filantropis Rose Elizabeth Fitzgerald","[Men, ur, ut, Ġte, ks, ,, ĠJoseph, ĠPatrick, Ġ"", Joe, "", ĠKennedy, ,, ĠSr, ., Ġadalah, Ġay, ah, ĠJohn, ĠFitzgerald, ĠKennedy, ,, Ġsed, angkan, ĠRose, ĠElizabeth, ĠFitzgerald, Ġadalah, Ġib, u, ., <|eot_id|>]","[,, "", Kennedy, Fitzgerald, .]"


### Memahami kode retrieve_correct_token_logprobs

#### llama_32_1B_50_pairs (OLD version)

In [3]:
results = pd.read_json("llama_32_1B_50_pairs.json")
row = results.loc[[46], :].copy()
row[["id", "question", "generated_completion", "answer"]]

,id,question,generated_completion,answer
46,8223462044041100059-0,"Dimana letak daerah ""Pardembanan""?","Maaf, saya tidak bisa memberikan informasi tentang lokasi ""Pardembanan"" karena tidak ada informasi yang lengkap tentang daerah tersebut. Namun, saya bisa memberikan informasi umum tentang Kabupaten Asahan,","Kabupaten Asahan, Sumatera Utara, Indonesia"


In [4]:
from retrieve_logprobs import retrieve_correct_token_logprobs
row_updated = retrieve_correct_token_logprobs(row, tokenizer)

In [6]:
row_updated['answer_logprobs']

46    {' Kab': -2.058924436569214, 'upaten': -3.707477662828751e-05, ' As': -6.55653229841846e-06, 'ahan': -2.145769485650817e-06, ',': -1.106772899627685, 'Sum': None, 'ater': None, 'a': None, 'Ut': None, 'ara': None, 'Ind': None, 'onesia': None}
Name: answer_logprobs, dtype: object

### ...